<a href="https://colab.research.google.com/github/Yuliya-HV/nlp-core/blob/main/Train_word_embeding_tfkeras_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 6.9 MB/s 
     |████████████████████████████████| 140 kB 61.1 MB/s 
     |████████████████████████████████| 1.1 MB 49.7 MB/s 
     |████████████████████████████████| 86 kB 7.0 MB/s 
     |████████████████████████████████| 212 kB 65.9 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 596 kB 55.9 MB/s 
     |████████████████████████████████| 127 kB 14.9 MB/s 
     |████████████████████████████████| 144 kB 52.0 MB/s 
     |████████████████████████████████| 271 kB 52.8 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 112 kB 41.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing ins

In [3]:
from datasets import load_dataset

In [1]:
import io
import glob
import numpy as np
import os
import regex as re
import pandas as pd
import string
import tqdm
import tensorflow as tf
from tensorflow.keras import layers
from collections import Counter

AUTOTUNE = tf.data.AUTOTUNESEED = 42
num_negative = 10


In [5]:
ds = load_dataset("zj88zj/PubMed_200k_RCT")

Using custom data configuration zj88zj--PubMed_200k_RCT-bf8d11840df6705f


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/zj88zj___text/zj88zj--PubMed_200k_RCT-bf8d11840df6705f/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2593169
    })
    test: Dataset({
        features: ['text'],
        num_rows: 34492
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 33941
    })
})

In [7]:
len(ds["train"])

2593169

In [8]:
def vocab_counter(data):
  vocab = Counter()
  for row in data:
    for word in str(row).split():
      vocab[word] += 1
  return vocab

In [9]:
vocab = vocab_counter(ds["train"])

In [10]:
vocab_len = len(vocab)
vocab_len

752062

In [11]:
df = pd.DataFrame(ds["train"])
df.describe()

,text
count,2593169
unique,2374863
top,
freq,190654


In [12]:
df["text_len"] = df["text"].apply(lambda x: len(str(x)))
df["text_len"].mean

<bound method NDFrame._add_numeric_operations.<locals>.mean of 0           11
1          237
2          171
3          184
4           98
          ... 
2593164    185
2593165     69
2593166     80
2593167     50
2593168      0
Name: text_len, Length: 2593169, dtype: int64>

In [13]:
int(df["text_len"].mean()), int(df["text_len"].max())

(136, 1821)

In [14]:
seq_len = int(df["text_len"].mean())

In [15]:
df = df.drop("text_len", 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [16]:
df.dropna(inplace=True)

In [16]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
file = "/content/gdrive/My Drive/data/med2/" + "file_{}.txt"
n = 0
for i, row in df.iterrows():
  with open(file.format(n), "w") as f:
    f.write(str(row["text"]))
    n += 1
       

In [19]:
path = "/content/gdrive/My Drive/data/med2/"

os.chdir(path)
file_list = []
for file in os.listdir():
  file_list.append(path + '/' + file)
file_list[:5]

['/content/gdrive/My Drive/data/med2//file_121630.txt',
 '/content/gdrive/My Drive/data/med2//file_121631.txt',
 '/content/gdrive/My Drive/data/med2//file_121632.txt',
 '/content/gdrive/My Drive/data/med2//file_121633.txt',
 '/content/gdrive/My Drive/data/med2//file_121634.txt']

In [21]:
train_data = tf.data.TextLineDataset(file_list)

In [22]:
# train_data = tf.data.Dataset.from_tensor_slices(pd.DataFrame(ds["test"]).to_dict(orient="list"))

In [23]:
def preprocessing(data):
  data = tf.strings.lower(data)
  return tf.strings.regex_replace(data, '[%s]'% re.escape(string.punctuation), "")


In [24]:
Vectorizer = layers.TextVectorization(
    standardize=preprocessing,
    max_tokens=32768,
    output_mode="int",
    output_sequence_length=seq_len
)

In [25]:
seq_len=136

In [ ]:
Vectorizer.adapt(train_data.batch(1024))

vocab_vectorized = Vectorizer.get_vocabulary()
print(vocab_vectorized[:3])

In [26]:
train_data_vectorized = train_data.batch(1024).prefetch(AUTOTUNE).map(Vectorizer).unbatch()

In [27]:
seqs = list(train_data_vectorized.as_numpy_iterator())
print(len(seqs))

29682


In [28]:
for seq in seqs[:10]:
  print(f"{seq} :: {[vocab_vectorized[i] for i in seq]}")

[26976     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0] :: ['24486335', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [29]:
def data_generator_sg(seqs, window, num_negative, vocab_size, seed):
  targets, contexts, labels = [], [], []
  sampling = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  for seq in tqdm.tqdm(seqs):
    positive_sg, _ = tf.keras.preprocessing.sequence.skipgrams(
        seq,
        vocabulary_size=vocab_size,
        sampling_table=sampling,
        window_size=window,
        negative_samples=0
    )

    for target_word, context_word in positive_sg:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1
      )
      neg_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_negative,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="neg_sampling"
      )

      neg_sampling_candidates = tf.expand_dims(
          neg_sampling_candidates, 1
      )

      context = tf.concat([context_class, neg_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_negative, dtype="int64")

      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [30]:
targets, contexts, labels = data_generator_sg(
    seqs=seqs,
    window=5,
    num_negative=num_negative,
    vocab_size=vocab_len,
    seed=42
)

100%|██████████| 29682/29682 [18:01<00:00, 27.45it/s]


In [31]:
targets, contexts, labels = np.array(targets), np.array(contexts)[:,:,0], np.array(labels)


In [32]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 11), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 11), dtype=tf.int64, name=None))>


In [33]:
class Word2Vec(tf.keras.Model):

  def __init__(self, vocab_size, emb_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size, 
                                             emb_dim, 
                                             input_length=1,
                                             name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                              emb_dim,
                                              input_length=num_negative+1)
    
  def call(self, pair):
    target, context = pair
    if len(target.shape) == 2:
      target = tf.sqeeze(target, axis=1)
    
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    return dots

In [34]:
def custom_loss(x_logit, y_true):
  return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [35]:
embedding_dim = 128
word2vec = Word2Vec(vocab_len, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy']
                 )


In [36]:
tensorbd_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
 

In [37]:
word2vec.fit(dataset, epochs=10, callbacks=[tensorbd_callback])

Epoch 1/10
1134/1134 [==============================] - 13s 9ms/step - loss: 2.2988 - accuracy: 0.1724
Epoch 2/10
1134/1134 [==============================] - 10s 9ms/step - loss: 2.0828 - accuracy: 0.2706
Epoch 3/10
1134/1134 [==============================] - 10s 9ms/step - loss: 1.8598 - accuracy: 0.3681
Epoch 4/10
1134/1134 [==============================] - 10s 9ms/step - loss: 1.6459 - accuracy: 0.4504
Epoch 5/10
1134/1134 [==============================] - 10s 9ms/step - loss: 1.4559 - accuracy: 0.5202
Epoch 6/10
1134/1134 [==============================] - 10s 9ms/step - loss: 1.2955 - accuracy: 0.5785
Epoch 7/10
1134/1134 [==============================] - 10s 9ms/step - loss: 1.1631 - accuracy: 0.6254
Epoch 8/10
1134/1134 [==============================] - 10s 9ms/step - loss: 1.0546 - accuracy: 0.6628
Epoch 9/10
1134/1134 [==============================] - 10s 9ms/step - loss: 0.9657 - accuracy: 0.6922
Epoch 10/10
1134/1134 [==============================] - 10s 9ms/step - l

In [38]:
weights = word2vec.get_layer("w2v_embedding").get_weights()[0]
vocab = Vectorizer.get_vocabulary()

In [41]:
path = "/content/gdrive/My Drive/data/"
out_v = io.open("vectors.tsv", "w", encoding="utf-8")
out_meta = io.open("metadata.tsv", "w", encoding="utf-8")

for i, word in enumerate(vocab):
  if i == 0:
    continue # skip 0 it is a padding
  vec = weights[i]
  out_v.write("\t".join([str(x) for x in vec]) + "\n")
  out_meta.write(word+"\n")
out_v.close()
out_meta.close()


In [43]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>